Average Weekly Earnings: We extract [Weekly Earnings (KAB9)] of the [Whole Economy]
    Observations start: January 2000

CPIH Index: Base year is 2015 (2015=100)
    Obsevations start: 2000 JAN

Productivity: [UK Whole Economy: Output per hour worked SA: Index 2022 = 100] -- this is the column we extract from the dataset since it is in index form
    Observations start: Q1 2000

In [3]:
import pandas as pd

In [65]:
xls = pd.ExcelFile("data/raw/awe.xls")
print(xls.sheet_names)

#Skip the first 8 rows which contain title and text
awe = pd.read_excel("data/raw/awe.xls", sheet_name = "1. AWE Total Pay", skiprows=9, header=None)
#Skip the first 8 rows which contain title and text
cpi = pd.read_csv("data/raw/cpi_index.csv", skiprows=8, header=None)
prod = pd.read_excel("data/raw/prdy.xlsx")

# Filtering columns of Average Weekly Earnings -- the dataset starts from January 2000
awe = awe[[0,1]]
awe.columns = ['Date', 'WeeklyEarn']

#Drops rows where Weekly Earnings column is empty -- used for some empty text cells
awe.dropna(subset=['WeeklyEarn'], inplace=True)

# Filtering columns of CPIH dataset -- starting from January 2000
cpi = cpi[[0,1]]
cpi.columns = ['Date', 'CPIH']

start_idx = cpi.loc[cpi['Date'].str.startswith('2000 JAN')].index[0]
cpi = cpi.loc[start_idx:].reset_index(drop=True)


# Filtering columns of productivity dataset - starting from Q1 of 2000
prod = prod[['Title', 'UK Whole Economy: Output per hour worked SA: Index 2022 = 100']]
prod = prod.rename(columns={'Title': 'Date',
                            'UK Whole Economy: Output per hour worked SA: Index 2022 = 100': 'OutputHR'})

start_idx = prod.loc[prod['Date'].str.startswith('2000 Q1')].index[0]
prod = prod.loc[start_idx:].reset_index(drop=True)

['Introduction', '1. AWE Total Pay', '2. AWE Bonus Pay', '3. AWE Regular Pay', '4. AWE Total Pay Index', '5. AWE Regular Pay Index', '6. Real AWE', 'Sampling Variability']


C:\Users\andre\PycharmProjects\uk_inflation_wage_project\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [69]:
# We expect 1-to-1 for awe and cpi, call shape to confirm
print(cpi.shape)
print(awe.shape)

# We call tail to see why there is one extra observation -- CPI has a September observation
print(cpi.tail())
print(awe.tail())

# Dropping last observation
cpi = cpi[0:-1]

# Converting both datasets to datetime
cpi['Date'] = pd.to_datetime(cpi['Date'], format = '%Y %b')
awe['Date'] = pd.to_datetime(awe['Date'])

# Converting columns to Month objects
cpi['Month'] = cpi['Date'].dt.to_period('M')
awe['Month'] = awe['Date'].dt.to_period('M')

# Merging on month
ts = pd.merge(cpi, awe, how='inner', on=['Month'])

# Dropping unused date columns and setting index to Month
ts.drop(columns=['Date_x', 'Date_y'], inplace=True)
ts.set_index('Month', inplace=True)

ts.head()


(306, 3)
(308, 3)
          Date   CPIH    Month
301 2025-02-01  135.6  2025-02
302 2025-03-01  136.1  2025-03
303 2025-04-01  137.7  2025-04
304 2025-05-01  138.0  2025-05
305 2025-06-01  138.4  2025-06
          Date  WeeklyEarn    Month
303 2025-04-01  719.763788  2025-04
304 2025-05-01  722.569363  2025-05
305 2025-06-01  724.738409  2025-06
306 2025-07-01  728.291840  2025-07
307 2025-08-01  732.815546  2025-08


,CPIH,WeeklyEarn
Month,,
2000-01,72.6,304.722925
2000-02,72.8,299.818577
2000-03,73.0,311.418715
2000-04,73.3,308.748756
2000-05,73.5,311.178774
